<a href="https://www.kaggle.com/archana060189/yelp-restaurant-recommendation-knn-svd?scriptVersionId=87996735" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


In [2]:
businesses = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=100000)
reviews = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json", lines=True, orient='columns', chunksize=100000)

In [3]:
for business in businesses:
    subset_business = business
    break
    
for review in reviews:
    subset_review = review
    break

In [4]:
subset_business.shape

(100000, 14)

In [5]:
subset_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [6]:
subset_review.shape

(100000, 9)

In [7]:
subset_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [8]:
subset_business.groupby(['city'])['review_count'].sum().sort_values()

city
Watertown,             5
El Cerrito             5
Atlanda                5
Boston College         5
Bolder                 5
                   ...  
Orlando           386718
Boston            450027
Atlanta           532026
Portland          683703
Austin            864466
Name: review_count, Length: 686, dtype: int64

In [9]:
city = subset_business[(subset_business['city'] == 'Austin')]
austin = city[['business_id','name','address', 'categories', 'attributes','stars']]
austin

,business_id,name,address,categories,attributes,stars
8,N3_Gs3DnX4k9SgpwJxdEfw,Lane Wells Jewelry Repair,"7801 N Lamar Blvd, Ste A140","Shopping, Jewelry Repair, Appraisal Services, ...","{'RestaurantsPriceRange2': '1', 'ByAppointment...",5.0
9,tXvdYGvlEceDljN8gt2_3Q,Capital City Barber Shop,"615 W Slaughter Ln, Ste 113","Barbers, Beauty & Spas","{'BusinessAcceptsCreditCards': 'False', 'Resta...",4.0
24,nTIhpR7MhsALPwg_Hh14EA,DoubleTree by Hilton Hotel Austin,6505 N Interstate 35,"Hotels, Hotels & Travel, Event Planning & Serv...","{'WiFi': 'u'free'', 'RestaurantsPriceRange2': ...",3.0
27,8XyEpVdAO0o6iVkVxkWosQ,PS Property Management Company,"2506 S Lamar Blvd, Ste 2","Home Services, Real Estate, Property Management",{'BusinessAcceptsCreditCards': 'True'},4.5
37,NVfOn7TdnHbaGH97CVB_Qg,McKinley Chiropractic,"5625 Eiger Rd, Ste 160","Chiropractors, Health & Medical","{'BusinessAcceptsCreditCards': 'True', 'Accept...",5.0
...,...,...,...,...,...,...
99988,HnCJvdFI05ED0H4q0o4qKA,Scott & Company Residential Renovations,,"Home Services, Flooring, Contractors, Painters","{'ByAppointmentOnly': 'True', 'BusinessAccepts...",5.0
99990,ErcLfvaM4Aef8GbmgKXJUw,Twin Liquors,"7301 N Fm 620, Ste 210","Food, Shopping, Tobacco Shops, Beer, Wine & Sp...","{'BusinessParking': '{'garage': False, 'street...",3.5
99993,WJLVOpQZ6mes3f0xfsiZ4g,Glow Nails Beauty Bar + Lounge,2947 W Anderson Ln,"Beauty & Spas, Nail Salons","{'BusinessAcceptsCreditCards': 'True', 'WiFi':...",5.0
99994,q6nowDVTOvusDFwWHumZYg,Bat City Gelato,"6301 W Parmer Ln, Ste 503","Gelato, Food, Ice Cream & Frozen Yogurt, Desserts","{'WheelchairAccessible': 'True', 'RestaurantsP...",5.0


# Data Preprocessing

In [10]:
rest = austin[austin['categories'].str.contains('Restaurant.*')==True].reset_index()
rest

,index,business_id,name,address,categories,attributes,stars
0,38,Xw8tuI30T-xihpzwBV-zJg,El Pollo Rey,1725 E Riverside Dr,"Food Trucks, Restaurants, Specialty Food, Food...","{'RestaurantsGoodForGroups': 'True', 'Restaura...",5.0
1,41,NRPemqVb4qpWFF0Avq_6OQ,Eurasia Sushi Bar & Seafood,"7101 W Hwy 71, Ste C-13","Bars, Nightlife, Cocktail Bars, Seafood, Resta...","{'Ambience': '{'touristy': False, 'hipster': F...",4.5
2,61,bRsDZ44CD3uhGnRY3NeQhQ,Wendy's,6247 Mcneil Drive,"Fast Food, Restaurants, Burgers","{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...",2.0
3,65,z-0oY7VxQMQw3JHvdPejrA,Olmecas Mexican Restaurant,2121 E Oltorf St,"Coffee & Tea, Tex-Mex, Restaurants, Mexican, Food","{'Ambience': '{'romantic': False, 'intimate': ...",3.0
4,108,qTdz29Nf3zoALiDDEKfWlw,Pho Oanh,"2121 E Oltorf St, Ste 12B","Taiwanese, Vietnamese, Restaurants","{'Ambience': '{'romantic': False, 'intimate': ...",3.5
...,...,...,...,...,...,...,...
3068,99832,8A9bsd1S3INHzYok3WeJJw,New Mandarin Chinese Restaurant,212 E Oltorf St,"Dim Sum, Chinese, Restaurants","{'RestaurantsGoodForGroups': 'True', 'Caters':...",2.5
3069,99858,CgxFVPWvfMjhE3F34OI9rw,Manny's Uptown Kitchen,"9503 Research Blvd, Ste 650","Restaurants, Diners, American (New)","{'RestaurantsDelivery': 'False', 'RestaurantsP...",3.5
3070,99911,yrHMqDwZf9iu3xLS11DK5A,The Omelettry,4631 Airport Blvd,"Diners, Restaurants, Breakfast & Brunch, Ameri...","{'NoiseLevel': ''average'', 'RestaurantsReserv...",3.5
3071,99973,-a0A1IKFVojTU7LUdbkwnA,Fran's Hamburgers,1822 S Congress Ave,"Burgers, Restaurants, Ice Cream & Frozen Yogur...","{'RestaurantsTakeOut': 'True', 'GoodForKids': ...",3.0


In [11]:
rest["attributes"][0]

{'RestaurantsGoodForGroups': 'True',
 'RestaurantsTakeOut': 'True',
 'HasTV': 'False',
 'RestaurantsReservations': 'False',
 'OutdoorSeating': 'True',
 'RestaurantsDelivery': 'False',
 'NoiseLevel': "'loud'",
 'WiFi': "'no'",
 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}",
 'BikeParking': 'True',
 'GoodForKids': 'True',
 'RestaurantsPriceRange2': '1',
 'RestaurantsAttire': "u'casual'",
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
 'BusinessAcceptsCreditCards': 'True'}

In [12]:
# Function that extract keys from the nested dictionary
def extract_keys(attr, key):
    if attr == None:
        return "{}"
    if key in attr:
        return attr.pop(key)

# convert string to dictionary
import ast
def str_to_dict(attr):
    if attr != None:
        return ast.literal_eval(attr)
    else:
        return ast.literal_eval("{}") 


In [13]:
str_to_dict(extract_keys({'RestaurantsGoodForGroups': 'True',
 'RestaurantsTakeOut': 'True',
 'HasTV': 'False',
 'RestaurantsReservations': 'False',
 'OutdoorSeating': 'True',
 'RestaurantsDelivery': 'False',
 'NoiseLevel': "'loud'",
 'WiFi': "'no'",
 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}",
 'BikeParking': 'True',
 'GoodForKids': 'True',
 'RestaurantsPriceRange2': '1',
 'RestaurantsAttire': "u'casual'",
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",
 'BusinessAcceptsCreditCards': 'True'}, "BusinessParking"))

{'garage': False,
 'street': False,
 'validated': False,
 'lot': False,
 'valet': False}

In [14]:
# get dummies from nested attributes
rest['BusinessParking'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'BusinessParking')), axis=1)
rest['Ambience'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Ambience')), axis=1)
rest['GoodForMeal'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'GoodForMeal')), axis=1)
rest['Dietary'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Dietary')), axis=1)
rest['Music'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Music')), axis=1)

In [15]:
rest

,index,business_id,name,address,categories,attributes,stars,BusinessParking,Ambience,GoodForMeal,Dietary,Music
0,38,Xw8tuI30T-xihpzwBV-zJg,El Pollo Rey,1725 E Riverside Dr,"Food Trucks, Restaurants, Specialty Food, Food...","{'RestaurantsGoodForGroups': 'True', 'Restaura...",5.0,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'classy...",{},{},{}
1,41,NRPemqVb4qpWFF0Avq_6OQ,Eurasia Sushi Bar & Seafood,"7101 W Hwy 71, Ste C-13","Bars, Nightlife, Cocktail Bars, Seafood, Resta...","{'CoatCheck': 'False', 'GoodForDancing': 'Fals...",4.5,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...","{'dessert': None, 'latenight': False, 'lunch':...",{},"{'dj': False, 'background_music': False, 'no_m..."
2,61,bRsDZ44CD3uhGnRY3NeQhQ,Wendy's,6247 Mcneil Drive,"Fast Food, Restaurants, Burgers","{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...",2.0,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...","{'dessert': False, 'latenight': False, 'lunch'...",{},{}
3,65,z-0oY7VxQMQw3JHvdPejrA,Olmecas Mexican Restaurant,2121 E Oltorf St,"Coffee & Tea, Tex-Mex, Restaurants, Mexican, Food","{'RestaurantsAttire': ''casual'', 'BYOBCorkage...",3.0,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'classy...",{},{},{}
4,108,qTdz29Nf3zoALiDDEKfWlw,Pho Oanh,"2121 E Oltorf St, Ste 12B","Taiwanese, Vietnamese, Restaurants","{'GoodForKids': 'True', 'RestaurantsReservatio...",3.5,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...
3068,99832,8A9bsd1S3INHzYok3WeJJw,New Mandarin Chinese Restaurant,212 E Oltorf St,"Dim Sum, Chinese, Restaurants","{'RestaurantsGoodForGroups': 'True', 'Caters':...",2.5,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...","{'dessert': False, 'latenight': False, 'lunch'...",{},{}
3069,99858,CgxFVPWvfMjhE3F34OI9rw,Manny's Uptown Kitchen,"9503 Research Blvd, Ste 650","Restaurants, Diners, American (New)","{'RestaurantsDelivery': 'False', 'RestaurantsP...",3.5,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",{},{},{}
3070,99911,yrHMqDwZf9iu3xLS11DK5A,The Omelettry,4631 Airport Blvd,"Diners, Restaurants, Breakfast & Brunch, Ameri...","{'NoiseLevel': ''average'', 'RestaurantsReserv...",3.5,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",{},{}
3071,99973,-a0A1IKFVojTU7LUdbkwnA,Fran's Hamburgers,1822 S Congress Ave,"Burgers, Restaurants, Ice Cream & Frozen Yogur...","{'RestaurantsTakeOut': 'True', 'GoodForKids': ...",3.0,"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",{},{},{}


In [16]:
# create table with attribute dummies
df_attr = pd.concat([ rest['attributes'].apply(pd.Series), rest['BusinessParking'].apply(pd.Series),
                    rest['Ambience'].apply(pd.Series), rest['GoodForMeal'].apply(pd.Series), 
                    rest['Dietary'].apply(pd.Series) ], axis=1)
df_attr_dummies = pd.get_dummies(df_attr)
df_attr_dummies

,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,HasTV_False,HasTV_None,HasTV_True,RestaurantsReservations_False,...,latenight_False,latenight_True,lunch_False,lunch_True,dinner_False,dinner_True,brunch_False,brunch_True,breakfast_False,breakfast_True
0,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,1,0,0,1,0
2,0,0,1,0,0,1,0,0,1,1,...,1,0,1,0,1,0,1,0,1,0
3,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,0,0,1,0,0,1,0,0,1,1,...,1,0,0,1,0,1,1,0,1,0
3069,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3070,0,0,1,0,0,1,1,0,0,1,...,1,0,0,1,1,0,0,1,0,1
3071,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# get dummies from categories
df_categories_dummies = pd.Series(rest['categories']).str.get_dummies(',')
df_categories_dummies

,Acai Bowls,Accessories,Active Life,Adult Entertainment,Afghan,African,Amateur Sports Teams,American (New),American (Traditional),Amusement Parks,...,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Wholesale Stores,Wine Bars,Wine Tasting Room,Wraps
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3069,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3070,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# pull out names and stars from rest table 
result = rest[['name','stars']]
result

,name,stars
0,El Pollo Rey,5.0
1,Eurasia Sushi Bar & Seafood,4.5
2,Wendy's,2.0
3,Olmecas Mexican Restaurant,3.0
4,Pho Oanh,3.5
...,...,...
3068,New Mandarin Chinese Restaurant,2.5
3069,Manny's Uptown Kitchen,3.5
3070,The Omelettry,3.5
3071,Fran's Hamburgers,3.0


In [19]:
df_final = pd.concat([df_attr_dummies, df_categories_dummies, result], axis=1)
df_final.drop('Restaurants',inplace=True,axis=1)
df_final

,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,HasTV_False,HasTV_None,HasTV_True,RestaurantsReservations_False,...,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Wholesale Stores,Wine Bars,Wine Tasting Room,Wraps,name,stars
0,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,El Pollo Rey,5.0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,Eurasia Sushi Bar & Seafood,4.5
2,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Wendy's,2.0
3,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Olmecas Mexican Restaurant,3.0
4,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Pho Oanh,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,New Mandarin Chinese Restaurant,2.5
3069,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Manny's Uptown Kitchen,3.5
3070,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,The Omelettry,3.5
3071,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Fran's Hamburgers,3.0


In [20]:
mapper = {1.0:1,1.5:2, 2.0:2, 2.5:3, 3.0:3, 3.5:4, 4.0:4, 4.5:5, 5.0:5}
df_final['stars'] = df_final['stars'].map(mapper)

In [21]:
df_final

,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,HasTV_False,HasTV_None,HasTV_True,RestaurantsReservations_False,...,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Wholesale Stores,Wine Bars,Wine Tasting Room,Wraps,name,stars
0,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,El Pollo Rey,5
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,Eurasia Sushi Bar & Seafood,5
2,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Wendy's,2
3,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Olmecas Mexican Restaurant,3
4,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Pho Oanh,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,New Mandarin Chinese Restaurant,3
3069,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Manny's Uptown Kitchen,4
3070,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,The Omelettry,4
3071,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Fran's Hamburgers,3


In [22]:
X = df_final.iloc[:,:-2]
y = df_final['stars']

# Content Based Recommendation- KNN

In [23]:
from sklearn.model_selection import train_test_split
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X, y, test_size=0.2, random_state=1)

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train_knn, y_train_knn)

y_pred = knn.predict(X_test_knn)

accuracy_train = knn.score(X_train_knn, y_train_knn)
accuracy_test = knn.score(X_test_knn, y_test_knn)

print(f"Score on training set: {accuracy_train}")
print(f"Score on test set: {accuracy_test}")

Score on training set: 0.596826688364524
Score on test set: 0.5447154471544715


In [25]:
display(df_final.iloc[-1:])

,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,HasTV_False,HasTV_None,HasTV_True,RestaurantsReservations_False,...,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Wholesale Stores,Wine Bars,Wine Tasting Room,Wraps,name,stars
3072,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,Chi Chinese Buffet,2


In [26]:
print("Validation set (Restaurant name): ", df_final['name'].values[-1])

Validation set (Restaurant name):  Chi Chinese Buffet


In [27]:
test_set = df_final.iloc[-1:,:-2]

test_set

,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,HasTV_False,HasTV_None,HasTV_True,RestaurantsReservations_False,...,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Wholesale Stores,Wine Bars,Wine Tasting Room,Wraps
3072,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
X_val =  df_final.iloc[:-1,:-2]
X_val

,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,HasTV_False,HasTV_None,HasTV_True,RestaurantsReservations_False,...,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Wholesale Stores,Wine Bars,Wine Tasting Room,Wraps
0,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3068,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3069,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3070,0,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
y_val = df_final['stars'].iloc[:-1]
y_val

0       5
1       5
2       2
3       3
4       4
       ..
3067    4
3068    3
3069    4
3070    4
3071    3
Name: stars, Length: 3072, dtype: int64

In [30]:
# test set from the df_final table (only last row): Restaurant name: "Chi Chinese Buffet"
test_set = df_final.iloc[-1:,:-2]

# validation set from the df_final table (exclude the last row)
X_val =  df_final.iloc[:-1,:-2]
y_val = df_final['stars'].iloc[:-1]

In [31]:
n_knn = knn.fit(X_val, y_val)

In [32]:
distances, indices =  n_knn.kneighbors(test_set)
n_knn.kneighbors(test_set)[0][0]
n_knn.kneighbors(test_set)[1][0]

array([1852, 1262, 1932, 1243,  331,  799, 2586, 1175, 1620,  590, 1916,
       2757, 1069,  284, 2046,  937,  400, 1002,  830,  732])

In [33]:
# distances and indeces from validation set (Chi Chinese Buffet)
distances, indices =  n_knn.kneighbors(test_set)
#n_knn.kneighbors(test_set)[1][0]

# create table distances and indeces from "Chi Chinese Buffet"
final_table = pd.DataFrame(n_knn.kneighbors(test_set)[0][0], columns = ['distance'])
final_table['index'] = n_knn.kneighbors(test_set)[1][0]
final_table.set_index('index')

,distance
index,
1852,3.000000
1262,3.000000
1932,3.605551
1243,3.741657
331,3.741657
799,3.872983
2586,3.872983
1175,3.872983
1620,3.872983


In [34]:
# get names of the restaurant that similar to the "Chi Chinese Buffet"
result = final_table.join(df_final,on='index')
result[['distance','index','name','stars']].head(5)

,distance,index,name,stars
0,3.000000,1852,Fujian Grand China Buffet,2
1,3.000000,1262,Los Pinos Mexican,4
2,3.605551,1932,Hao-Q Asian Kitchen,4
3,3.741657,1243,A-OK Chinese,3
4,3.741657,331,Mr Catfish,4


# Collaborative filtering - SVD

In [35]:
subset_review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [36]:
# pull out needed columns from subset_review table
df_review = subset_review[['user_id','business_id','stars', 'date']]
df_review

,user_id,business_id,stars,date
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,2014-10-11 03:34:02
1,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,2015-07-03 20:38:25
2,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,2013-05-28 20:38:06
3,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2,2010-01-08 02:29:15
4,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4,2011-07-28 18:05:01
...,...,...,...,...
99995,wCsUDt0ZgcTVXj3oyiqmpw,3cLBtLFiH8IRdlEy9S8RiQ,2,2017-08-31 15:54:00
99996,XYpKK63VKDgqgKfltDtuoQ,1V5AWwFsxrlUsSo2B6MEEA,4,2011-04-27 14:37:44
99997,Bf03D8paZqH0FWPBN_5yQg,w4vP6vhfb65Te66zOO2Hug,5,2017-10-27 01:17:18
99998,c9TXAfOlLeTTteGI_VV0Og,8u_pdcdN66D1dmIZrNK55w,4,2012-07-01 17:24:39


In [37]:
# pull out names and addresses of the restaurants from rest table
restaurant = rest[['business_id', 'name', 'address']]
restaurant

,business_id,name,address
0,Xw8tuI30T-xihpzwBV-zJg,El Pollo Rey,1725 E Riverside Dr
1,NRPemqVb4qpWFF0Avq_6OQ,Eurasia Sushi Bar & Seafood,"7101 W Hwy 71, Ste C-13"
2,bRsDZ44CD3uhGnRY3NeQhQ,Wendy's,6247 Mcneil Drive
3,z-0oY7VxQMQw3JHvdPejrA,Olmecas Mexican Restaurant,2121 E Oltorf St
4,qTdz29Nf3zoALiDDEKfWlw,Pho Oanh,"2121 E Oltorf St, Ste 12B"
...,...,...,...
3068,8A9bsd1S3INHzYok3WeJJw,New Mandarin Chinese Restaurant,212 E Oltorf St
3069,CgxFVPWvfMjhE3F34OI9rw,Manny's Uptown Kitchen,"9503 Research Blvd, Ste 650"
3070,yrHMqDwZf9iu3xLS11DK5A,The Omelettry,4631 Airport Blvd
3071,-a0A1IKFVojTU7LUdbkwnA,Fran's Hamburgers,1822 S Congress Ave


In [38]:
combined_business_data = pd.merge(df_review, restaurant, on='business_id')
combined_business_data

,user_id,business_id,stars,date,name,address
0,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,2017-05-26 03:05:46,Pho Van,8557 Research Blvd
1,ZuB2N9UOIooj9nPtI3Re5g,mOnesB4IF9j6-ZmHoOHOig,2,2014-01-17 00:55:26,Pho Van,8557 Research Blvd
2,AQUdnD35RBuCXNx0KxE5wA,mOnesB4IF9j6-ZmHoOHOig,4,2015-06-21 00:30:24,Pho Van,8557 Research Blvd
3,IYRlxx4G8fkkc6x_gk4sPg,mOnesB4IF9j6-ZmHoOHOig,5,2017-09-08 23:20:38,Pho Van,8557 Research Blvd
4,ZYs9y2Hk_C55Ynr40TGmUA,mOnesB4IF9j6-ZmHoOHOig,4,2010-07-07 16:27:26,Pho Van,8557 Research Blvd
...,...,...,...,...,...,...
10941,RKc1JWXE0n9QnGOtYALy4Q,JmYpdmtF5MlHCnjacKOOFg,1,2017-03-18 03:09:22,Super Burrito East,1906 E Cesar Chavez St
10942,heAiS4gELyKc-pv6gpfxqA,o3fCnYYhmIvTH1MIruA1oA,5,2017-08-17 16:53:28,Green Lunch,
10943,eSB6dQONdp6P_wXwjZvPJQ,lZhlImFp2IQVFykpqodzYg,5,2013-01-05 19:31:56,Jo Ellen's Tacos,3815 Dry Creek Dr
10944,xq_0nxzdStXmQKTAEcSWsA,8JDbhZfA2cbrqswYTFHbvA,5,2012-09-12 20:14:25,Austin's Best Soup,


In [39]:
# the most POPULAR restaurants by stars.
combined_business_data.groupby('business_id')['stars'].count().sort_values(ascending=False).head()

business_id
bZiIIUcpgxh8mpKMDhdqbA    713
jREzLrIEkc4jQKLfYMJ0gg    370
v1UzkU8lEWdjxq8byWFOKg    298
YZs1gNSh_sN8JmN_nrpxeA    245
XNrAtQFU4vl9NA8GTlNonA    226
Name: stars, dtype: int64

In [40]:
# see the NAME of the most popular restaurant
Filter = combined_business_data['business_id'] == 'bZiIIUcpgxh8mpKMDhdqbA'
print("Name: ", combined_business_data[Filter]['name'].unique())
print("Address:", combined_business_data[Filter]['address'].unique())

Name:  ['Hopdoddy Burger Bar']
Address: ['1400 S Congress Ave, Ste A190']


In [41]:
# create a user-item matrix
rating_crosstab = combined_business_data.pivot_table(values='stars', index='user_id', columns='name', fill_value=0)
rating_crosstab.head()

name,1776 Cheesesteak,34th Street Cafe,360 Pizza,888 Pan Asian Restaurant,Al Forno Slice & Pies,Alamo Drafthouse Cinema Mueller,Alamo Drafthouse Cinema South Lamar,Alta's Cafe,Amy's Ice Creams,Andiamo Ristorante,...,Yummy Thai Food,Yuyo,ZPizza,Zack's American Bistro,Zaika Indian Contemporary Cuisine,Zen Japanese Food Fast,Zi Pizza,Zoes Kitchen,Zuzu Handmade Mexican Food,davids tacos
user_id,,,,,,,,,,,,,,,,,,,,,
-0-p3YrmDoJ9TL-vyVvU2A,0,0,4,0.0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
-0MQ4webH2uc1ZAsGsNENg,0,0,0,0.0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
-2gtQ6A_xdZ2a8Ce5TyRYQ,0,0,0,0.0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
-2hpnnC2AKW3CRn7Sra_3w,0,0,0,0.0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
-2sNTzGyci98Mp9PmPRg8w,0,0,0,0.0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
rating_crosstab.shape

(9358, 389)

In [43]:
X = rating_crosstab.values.T
X.shape

(389, 9358)

In [44]:
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score


SVD = TruncatedSVD(n_components=12, random_state=17)
result_matrix = SVD.fit_transform(X)
result_matrix.shape

(389, 12)

In [45]:
# PearsonR coef 
corr_matrix = np.corrcoef(result_matrix)
corr_matrix.shape

(389, 389)

In [46]:
# get the index of the popular restaurant
restaurant_names = rating_crosstab.columns
restaurants_list = list(restaurant_names)

popular_rest = restaurants_list.index('Hopdoddy Burger Bar')
print("index of the popular restaurant: ", popular_rest)

index of the popular restaurant:  159


In [47]:
# restaurant of interest 
corr_popular_rest = corr_matrix[popular_rest]
corr_popular_rest.shape 

(389,)

In [48]:
list(restaurant_names[(corr_popular_rest < 1.0) & (corr_popular_rest > 0.9)])

['1776 Cheesesteak',
 '360 Pizza',
 'Art of Tacos',
 "Hoek's Death Metal Pizza",
 "Luby's",
 'Mum Foods',
 'Starbucks',
 "Wahoo's Fish Tacos"]

In [49]:
display(rest[rest['name'] == 'Hopdoddy Burger Bar'])

,index,business_id,name,address,categories,attributes,stars,BusinessParking,Ambience,GoodForMeal,Dietary,Music
53,1850,bZiIIUcpgxh8mpKMDhdqbA,Hopdoddy Burger Bar,"1400 S Congress Ave, Ste A190","American (New), American (Traditional), Restau...","{'Alcohol': 'u'full_bar'', 'GoodForKids': 'Tru...",4.0,"{'garage': True, 'street': True, 'validated': ...","{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",{},{}
2752,88857,67bZm-KQQCwSat7_jmiVbA,Hopdoddy Burger Bar,"2438 W Anderson Ln, Ste100","Food, Restaurants, Nightlife, Bars, Juice Bars...","{'NoiseLevel': 'u'loud'', 'GoodForKids': 'True...",4.0,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",{},"{'dj': False, 'background_music': False, 'no_m..."
